##Part 1 - Data Ingestion and Preparation

1. Create 3 variables

In [0]:
storage_account_name = "boompeeranont"
storage_account_access_key = "upBPWiJll16gB28tjwWJghsZ79c5omrg0ewjk43EvvUtQGmoeeovplr+K15cHyn4nsWnKo7p54ib+AStw7lWbg=="
blob_container_name = "dummy-1"

2. Mount the Blob container to the DBFS

In [0]:
dbutils.fs.mount(
  source = f'wasbs://{blob_container_name}@{storage_account_name}.blob.core.windows.net',
  mount_point = f'/mnt/{blob_container_name}/',
  extra_configs = {'fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net': storage_account_access_key}
)

3. Make the list of files inside the blob

In [0]:
dbutils.fs.ls("/mnt/dummy-1/")


4. Read the dataset

In [0]:
#Read the Green taxi dataset
df_g = spark.read.option("header", True).parquet("/mnt/dummy-1/green_taxi*")
display(df_g)

In [0]:
#Read the Yellow taxi dataset
df_y = spark.read.option("header", True).parquet("/mnt/dummy-1/yellow_taxi*")
display(df_y)

In [0]:
df_g.printSchema()

In [0]:
df_y.printSchema()

In [0]:
#count row of Green and Yellow dataset
print("Green taxi row count:",df_g.count())
print("Yellow taxi row count:",df_y.count())

In [0]:
#Convert the green taxi 2015 from parquet to csv
#df_g_2015 = spark.read.parquet("/mnt/dummy-1/green_taxi_2015.parquet")
df_g_2015.write.mode("overwrite").option("header", True).csv("/mnt/dummy-1/csv_2015")

5. Cleaning the dataset

5.1 Green Taxi

In [0]:
#Explore the dataset
display(df_g)

In [0]:
print((df_g.count(), len(df_g.columns)))

In [0]:
import pyspark.sql.functions as F

In [0]:
df_g.printSchema()

In [0]:
#1) The Trips finishing before the starting time
filter_1 = df_g.filter(df_g['lpep_pickup_datetime'] > df_g['lpep_dropoff_datetime'])
display(filter_1)

In [0]:
#Count filter 1
filter_1.count()

In [0]:
#Filter 1) out
df_g_c = df_g.filter(df_g['lpep_pickup_datetime'] < df_g['lpep_dropoff_datetime'])

In [0]:
#Save file #1
df_g_c.write.mode("overwrite").option("header", True).parquet("/mnt/dummy-1/df_green_cleaned.parquet")

In [0]:
#Load file #1
df_g_c = spark.read.option("header", True).parquet("/mnt/dummy-1/df_green_cleaned.parquet")
display(df_g_c)

In [0]:
#Count the row after filtering out
df_g_c.cache().count()

In [0]:
#Copy dataframe to test some code

df_g_copy = df_g_c.select("*")
display(df_g_copy)


In [0]:
from pyspark.sql.functions import split, col

# Split the lpep_pickup_datetime column into date and time
df_g_copy = df_g_copy.withColumn("date_pickup", split(col("lpep_pickup_datetime"), "T").getItem(0))

df_g_copy = df_g_copy.withColumn("d_pickup", split(col("date_pickup"), " ").getItem(0))\
            .withColumn("t_pickup", split(col("date_pickup"), " ").getItem(1))
df_g_copy = df_g_copy.drop("date_pickup")
display(df_g_copy, truncate = False)


In [0]:
# Split the timestamp column into date and time
df_g_copy = df_g_copy.withColumn("date_dropoff", split(col("lpep_dropoff_datetime"), "T").getItem(0))

df_g_copy = df_g_copy.withColumn("d_dropoff", split(col("date_dropoff"), " ").getItem(0))\
            .withColumn("t_dropoff", split(col("date_dropoff"), " ").getItem(1))
df_g_copy = df_g_copy.drop("date_dropoff")
display(df_g_copy, truncate = False)


In [0]:
#When it is able to solve then make it become the real dataframe
df_g_c = df_g_copy
display(df_g_c)

In [0]:
# 2) Trips where the pickup/dropoff datetime is outside of the range
# Filter the out-of-range time
filter_2_1 = df_g_c.filter((df_g_c['t_pickup'] > '23:59:59') | (df_g_c['t_dropoff'] > '23:59:59')|(df_g_c['t_pickup'] < '00:00:00')|(df_g_c['t_dropoff'] < '00:00:00'))
display(filter_2_1)


In [0]:
from pyspark.sql.functions import to_date

# Convert the string columns to date 
df_g_c = df_g_c.withColumn("d_pickup", to_date(df_g_c["d_pickup"], "yyyy-MM-dd")) \
               .withColumn("d_dropoff", to_date(df_g_c["d_dropoff"], "yyyy-MM-dd"))


filter_2_2 = df_g_c.filter(
    
    (col("d_pickup") < "2015-01-01") | 
    (col("d_dropoff") < "2015-01-01")
)

display(filter_2_2)

In [0]:
#Filter filter_2_2 out
df_g_c = df_g_c.filter((col("d_pickup") >= "2015-01-01") & (col("d_pickup") <= "2022-12-31"))
display(df_g_c)

In [0]:
#Save file #2
df_g_c.write.mode("overwrite").option("header", True).parquet("/mnt/dummy-1/df_green_cleaned.parquet")

In [0]:
#Load file #2
df_g_c_2 = spark.read.option("header", True).parquet("/mnt/dummy-1/df_green_cleaned.parquet")
display(df_g_c_2)

In [0]:
df_g_c_2.count()

In [0]:
display(df_g_c_2.limit(5))

In [0]:
df_zone = spark.read.option("header", True).csv("/FileStore/tables/taxi_zone_lookup.csv")
display(df_zone)

In [0]:
df_g_c_2 = df_g_c_2.cache()

In [0]:
# 3) Trips with negative speed
from pyspark.sql.functions import unix_timestamp, round, col
# Calculate duration in seconds by subtracting pickup and dropoff with unix timestamp
df_g_c_2 = df_g_c_2.withColumn(
    "duration_seconds",
    unix_timestamp(col("lpep_dropoff_datetime")) - unix_timestamp(col("lpep_pickup_datetime"))
)
# Convert seconds to hours with 2 decimal points
df_g_c_2 = df_g_c_2.withColumn(
    "duration_hours",
    round(col("duration_seconds") / 3600, 2)
)


display(df_g_c_2.limit(10))

In [0]:
# Determine Speed of each trip with divided trip_distance by duration_hours
df_g_c_2 = df_g_c_2.withColumn(
    "trip_speed",
    round(col("trip_distance") / col("duration_hours"), 2)
)

display(df_g_c_2.limit(10))

In [0]:
df_g_c_2 = df_g_c_2.drop("trips_speed") # Duplicated column made by mistake
display(df_g_c_2.limit(10))

In [0]:
#Find any trips with negative speed
filter_3 = df_g_c_2.where("trip_speed < 0")
display(filter_3)

In [0]:
# Count the filter_3
filter_3.count()

In [0]:
#Filter the trips with negative speed out and count the rows have left
df_g_c_2 = df_g_c_2.filter(col("trip_speed") >= 0)
df_g_c_2.count()

In [0]:
# 4) Trips with very high speed (look for NYC and outside of NYC speed limit )
# NYC has 25 mph speed limit unless otherwise posted and outside NYC is 70 mph.
# The trip is speed over 25 mph that have to be filtered out because most of trips are in NYC.
filter_4 = df_g_c_2.where((col("trip_speed") > 25))
display(filter_4.select("PULocationID","DOLocationID","trip_distance", "duration_hours", "trip_speed"))


In [0]:
filter_4.count()

In [0]:
#Filter the trips with over speed limit and count the rows have left
df_g_c_2 = df_g_c_2.filter((col("trip_speed") < 25) & (col("trip_speed") >= 0))
df_g_c_2.count()

In [0]:
df_g_c_2.unpersist()

In [0]:
#Save file #3
df_g_c_2.write.mode("overwrite").parquet("/FileStore/tables/df_green_cleaned.parquet")

In [0]:
#Load file #3
df_g_c_3 = spark.read.option("header", True).parquet("/FileStore/tables/df_green_cleaned.parquet")
display(df_g_c_3)

In [0]:
df_g_c_3.count()

In [0]:
# 5) Trips that are travelling too short or too long (duration wise)
#Find the trips that are too short duration.
# Find trips that have duration shorter than 0.02 * 60 = 1.2 minutes
filter_5 = df_g_c_3.where("duration_hours < 0.02")
display(filter_5.select("duration_hours"))
filter_5.count()

In [0]:
#Filter 5) out
from pyspark.sql.functions import col
df_g_c_3 = df_g_c_3.filter(col("duration_hours") >= 0.02)
df_g_c_3.select("duration_hours").show()

In [0]:
df_g_c_3.count()

In [0]:
#Find the trips that too long
# A driver shift is 12 hours is from the rule. 
filter_5_1 = df_g_c_3.where("duration_hours > 12.00")
display(filter_5_1.select("duration_hours"))
filter_5_1.count()

In [0]:
#Filter 5_1 out
df_g_c_3 = df_g_c_3.filter(col("duration_hours") <= 12.00)
df_g_c_3.select("duration_hours").show()
df_g_c_3.count()

In [0]:
# 6) Trips that are travelling too short or too long (distance wise)
#Find the trips that are too short
#Filter the same Pickup and Dropoff location or distance shorter than 0.1 miles or 0.16 km which is the distance that regular people can walk.
filter_6 = df_g_c_3.where((col("trip_distance") < 0.1))
filter_6.select("PULocationID", "DOLocationID", "trip_distance").show()
filter_6.count()

In [0]:
#Filter 6) out
df_g_c_3 = df_g_c_3.filter(col("trip_distance") >= 0.1)
df_g_c_3.select("trip_distance").show()
df_g_c_3.count()

In [0]:
#Find the trips that are too long distance.
#Measured the distance from the northest to the southest of NYC is around 50 miles. So, taxi can't commute more than 50 miles
#Filter the trip that has the distance more than 50 miles
filter_6_1 = df_g_c_3.where((col("trip_distance") > 50))
filter_6_1.select("PULocationID", "DOLocationID", "trip_distance").show()
filter_6_1.count()


In [0]:
#Filter 6_1) out
df_g_c_3 = df_g_c_3.filter(col("trip_distance") <= 50)
df_g_c_3.select("trip_distance").show()
df_g_c_3.count()


In [0]:
#7) Any other logic you think is important
#Show the cleaned dataset of green taxi so far
display(df_g_c_3)
df_g_c_3.count()

In [0]:
#Use Spark SQL and create temp view for SQL
df_g_c_3.createOrReplaceTempView("taxi_g_trips")

In [0]:
#Filter the total amount charged to passengers less than 3 because the initial charge of 2015-2022 is $2.50 
filter_7_1 = spark.sql("SELECT total_amount FROM taxi_g_trips WHERE total_amount < 2.50")
filter_7_1.show()
filter_7_1.count()

In [0]:
#Filter 7_1) out
df_g_c_3 = df_g_c_3.filter("total_amount >= 2.50")
display(df_g_c_3)
df_g_c_3.count()

In [0]:
#Show the value of each MTA tax
filter_7_2 = spark.sql("SELECT mta_tax, COUNT(*) AS numbers FROM taxi_g_trips GROUP BY mta_tax ")
filter_7_2.show()

In [0]:
#Filter the MTA tax that is not 0.5 and 0 out because $0.5 MTA tax is applied to trips inside Manhattan, Other trip outside Manhattan MTA tax is not applied.
df_g_c_3 = df_g_c_3.filter((col("mta_tax") == 0) | (col("mta_tax") == 0.5))
display(df_g_c_3)
df_g_c_3.count()

In [0]:
#Filter the extra that occur when trips are driving in rush hour or overnight. 
filter_7_3 = spark.sql("SELECT extra, COUNT(*) AS numbers FROM taxi_g_trips WHERE extra IN (0, 0.5, 1) GROUP BY extra")
filter_7_3.show()

In [0]:
# Filter only 7_3 
df_g_c_3 = df_g_c_3.filter((col("extra") == 0) | (col("extra") == 0.5) | (col("extra") == 1.0))
display(df_g_c_3)
df_g_c_3.count()

In [0]:
#Find improvement surcharge how much of each trip.
filter_7_4 = spark.sql("SELECT improvement_surcharge, COUNT(*) AS numbers FROM taxi_g_trips GROUP BY improvement_surcharge")
filter_7_4.show()

In [0]:
# Filter 7_4 only Improvement_surcharge = 0.3 
df_g_c_3 = df_g_c_3.filter((col("Improvement_surcharge") == 0.3))
display(df_g_c_3)
df_g_c_3.count()

In [0]:
#Filter the negative value of fare_amount, tip_amount, tolls_amount
filter_7_5 = spark.sql("SELECT fare_amount, count(*) AS numbers FROM taxi_g_trips where fare_amount < 0 GROUP BY fare_amount ")
filter_7_5.show()
filter_7_5.count()

In [0]:
filter_7_6 = spark.sql("SELECT tip_amount, count(*) AS numbers FROM taxi_g_trips where tip_amount < 0 GROUP BY tip_amount ")
filter_7_6.show()
filter_7_6.count()

In [0]:
filter_7_7 = spark.sql("SELECT tolls_amount, count(*) AS numbers FROM taxi_g_trips where tolls_amount < 0 GROUP BY tolls_amount ")
filter_7_7.show()
filter_7_7.count()

In [0]:
#Filter 7_5, 7_6 and 7_7 out
df_g_c_3 = df_g_c_3.filter((col("fare_amount") >= 0) | (col("tolls_amount") >= 0) |(col("tip_amount") >= 0))
display(df_g_c_3)
df_g_c_3.count()

In [0]:
#Find the impossible passenger count
filter_7_8 = spark.sql("SELECT passenger_count, count(*) AS numbers FROM taxi_g_trips GROUP BY passenger_count ")
filter_7_8.show()


In [0]:
#Filter out the passenger count more than 5 passengers and 0 passenger because The maximum amount of passengers are allowed in a taxicab by law is four in a four passenger taxicab or five passengers in a five passenger taxicab.
df_g_c_3 = df_g_c_3.filter((col("passenger_count") <= 5) & (col("passenger_count") > 0))
display(df_g_c_3)
df_g_c_3.count()

In [0]:
#Drop the timestamp columns 
df_g_c_3 = df_g_c_3.drop("lpep_pickup_datetime", "lpep_dropoff_datetime", "duration_seconds")
display(df_g_c_3)

In [0]:
#Save the final green taxi dataframe 
df_g_c_3.write.mode("overwrite").parquet("/FileStore/tables/df_green_cleaned.parquet")

In [0]:
#Load the final green taxi dataframe
df_g_cleaned = spark.read.option("header", True).parquet("/FileStore/tables/df_green_cleaned.parquet")
display(df_g_cleaned)
df_g_cleaned.count()

In [0]:
# Load green taxi dataframe
df_g_cleaned = spark.read.option("header", True).parquet("/FileStore/tables/df_green_cleaned.parquet")
display(df_g_cleaned)
df_g_cleaned.count()

In [0]:
#Drop some unused columns 
df_g_cleaned = df_g_cleaned.drop("VendorID", "store_and_fwd_flag", "RatecodeID", "payment_type", "trip_type")
display(df_g_cleaned)

In [0]:
#Use Spark SQL and create temp view for SQL
df_g_cleaned.createOrReplaceTempView("taxi_g_trips")

In [0]:
#Filter the congestion surcharge
filter_cs = spark.sql("SELECT congestion_surcharge, count(*) AS numbers FROM taxi_g_trips GROUP BY congestion_surcharge")
filter_cs.show()

In [0]:
#Filter only 2.75, 0 and null value
df_g_cleaned = df_g_cleaned.filter((col("congestion_surcharge").isNull()) | (col("congestion_surcharge") == 0) | (col("congestion_surcharge") == 2.75))
display(df_g_cleaned)
df_g_cleaned.count()

In [0]:
#Save the final version of green taxi
df_g_cleaned.write.mode("overwrite").parquet("/FileStore/tables/df_green_cleaned.parquet")

In [0]:
df_g_cleaned = spark.read.option("header", True).parquet("/FileStore/tables/df_green_cleaned.parquet")
display(df_g_cleaned)
df_g_cleaned.count()